In [ ]:
#RESNET model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import time
import numpy as np
import random
import os

# ================================
# 1. Setup
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Fix random seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

data_dir = r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_5\Faulty_solar_panel"
batch_size = 32
num_classes = 6
EPOCHS = 50  # set higher, early stopping will cut it if needed
patience = 2  # stop if val_loss increases for 2 epochs

# ================================
# 2. Data Transforms & Loaders
# ================================
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform_train)
num_train = len(full_dataset)
val_size = int(0.2 * num_train)
train_size = num_train - val_size

train_set, val_set = random_split(full_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(seed))
val_set.dataset.transform = transform_val

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)

# ================================
# 3. Model: ResNet18 (pretrained)
# ================================
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Mixed precision scaler
scaler = torch.cuda.amp.GradScaler()

# ================================
# 4. Training & Validation Loop
# ================================
train_losses, val_losses, val_accs = [], [], []
best_val_loss = float("inf")
best_epoch = -1
early_stop_counter = 0

save_path = "best_resnet18_model.pth"

for epoch in range(EPOCHS):
    start_time = time.time()

    # ---- Training ----
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # ---- Validation ----
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total

    val_losses.append(avg_val_loss)
    val_accs.append(val_acc)

    elapsed = time.time() - start_time
    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Loss: {avg_train_loss:.4f} "
          f"Val Loss: {avg_val_loss:.4f} "
          f"Val Acc: {val_acc*100:.2f}% "
          f"(Time: {elapsed:.2f}s)")

    # ---- Early stopping & Model saving ----
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch
        early_stop_counter = 0
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved at epoch {epoch+1} with Val Loss: {avg_val_loss:.4f}")
    else:
        early_stop_counter += 1
        print(f"⚠️ Early stopping counter: {early_stop_counter}/{patience}")
        if early_stop_counter >= patience:
            print("⛔ Training stopped early due to no improvement in validation loss.")
            break

print(f"Best model was from epoch {best_epoch+1} with Val Loss: {best_val_loss:.4f}")
import numpy as np

np.save("val_losses.npy", np.array(val_losses))
np.save("train_losses.npy", np.array(train_losses))
np.save("val_accs.npy", np.array(val_accs))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load saved data
val_losses = np.load("val_losses.npy")
train_losses = np.load("train_losses.npy")
val_accs = np.load("val_accs.npy")

# --- Plot 1: Training vs Validation Loss ---
plt.figure(figsize=(8,6))
plt.plot(range(1, len(val_losses)+1), val_losses, marker='o', label="Validation Loss", color="red")
plt.plot(range(1, len(train_losses)+1), train_losses, marker='s', label="Training Loss", color="blue")
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Loss", fontsize=14)   # 🔹 increased font size for y-axis label
plt.title("Training vs Validation Loss", fontsize=14)
plt.legend()
plt.grid(True)
plt.show()

# --- Plot 2: Validation Accuracy ---
plt.figure(figsize=(8,6))
plt.plot(range(1, len(val_accs)+1), val_accs, marker='^', color="green", label="Validation Accuracy")
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Accuracy", fontsize=14)
plt.title("Validation Accuracy", fontsize=14)
plt.ylim(0.5, 1.0)   # 🔹 restrict y-axis from 0.5 to 1
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import time
import os

# ================================
# 1. Setup
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

num_classes = 6   # same as training
save_path = "best_resnet18_model.pth"

# Load model
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model.load_state_dict(torch.load(save_path, map_location=device))
model = model.to(device)
model.eval()

# ================================
# 2. Define transforms (same as validation)
# ================================
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ================================
# 3. Prediction function
# ================================
def predict_image(image_path, class_names):
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform_val(img).unsqueeze(0).to(device)

    start_time = time.time()
    with torch.no_grad():
        outputs = model(img_tensor)
        _, pred = torch.max(outputs, 1)
    elapsed = time.time() - start_time

    predicted_class = class_names[pred.item()]

    # Show image + prediction
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class}")
    plt.axis("off")
    plt.show()

    print(f"✅ Prediction: {predicted_class}")
    print(f"⏱️ Time taken: {elapsed:.4f} seconds")

# ================================
# 4. Run prediction
# ================================
# Load class names from dataset folder
data_dir = r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_5\Faulty_solar_panel"
class_names = os.listdir(data_dir)
class_names.sort()  # ensure same order as ImageFolder

# Ask user for input image
image_path = input("Enter the image path: ").strip('"')
predict_image(image_path, class_names)